# Muestreo de transacciones bancarias

### Asunciones
- La muestra es representativa cuando:
    - La distribución de sus valores es igual a la de la población.
    - No hay fuga de valores categóricos y se mantienen sus proporciones.
    - Se respetan los aspectos temporales:
        - Distrubución temporal de los valores se mantiene en todas las escalas.
        - Las transacciones periódicas (subscripciones, nóminas, etc.) mantienen su continuidad.

### Objetivos
- Diseñar y desarrollar un método para evaluar si una muestra es representativa del total.
- Diseñar y desarrollar un método para extraer muestras de tamaño N, y evaluar la calidad del muestreo para diferentes valores de N.

### Procedimiento
- Función de evaluación:
    - Recibe población y muestra.
    - Retorna una métrica de representatividad.
    - Funcionamiento:
        - Evalua los aspectos clave de todas las columnas del dataset procesado de la población:
            - Distrubución: normalidad, asimetría y curtosis.
            - 

In [126]:
import pandas as pd
import numpy as np
import plotly.express as px
import glob
import datetime
from pprint import pprint

pd.set_option("display.max_colwidth", 3000)

In [127]:
df = pd.concat([pd.read_csv(csv, sep=";") for csv in glob.glob("*.csv")])
# df = [pd.read_csv(csv, sep=";") for csv in glob.glob("*.csv")][1]

## Exploratory Data Analysis

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   amount     20000 non-null  float64
 1   date       20000 non-null  object 
 2   title      20000 non-null  object 
 3   Categoría  20000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 781.2+ KB


In [129]:
all_strings = all([isinstance(obj, str) for obj in df.date.iloc])
print(f"Column 'date': {all_strings=}")

Column 'date': all_strings=True


In [130]:
all_strings = all([isinstance(obj, str) for obj in df.title.iloc])
print(f"Column 'title': {all_strings=}")

Column 'title': all_strings=True


In [131]:
df.rename(columns={"Categoría": "category"}, inplace=True)
all_strings = all([isinstance(obj, str) for obj in df.category.iloc])
print(f"Column 'category': {all_strings=}")

Column 'category': all_strings=True


#### Columna 'date'

In [132]:
df['datetime'] = pd.to_datetime(df.date)

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second
df['utc_offset'] = df['datetime'].apply(lambda x: x.utcoffset().total_seconds())

In [133]:
seasons_map = {
    1 : 'winter',
    2 : 'winter',
    3 : 'spring',
    4 : 'spring',
    5 : 'spring',
    6 : 'summer',
    7 : 'summer',
    8 : 'summer',
    9 : 'fall',
    10 : 'fall',
    11 : 'fall',
    12 : 'winter',
}
df['season'] = df.month.replace(seasons_map)

In [134]:
df = df.drop(["date"], axis=1)

In [146]:
px.histogram(df[['season', 'month', 'amount']], x='month', y='amount', color='season')

In [137]:
px.histogram(df[['datetime', 'amount']].groupby('datetime', as_index=False).median(),x='datetime', y='amount', nbins = 365)

In [138]:
px.histogram(df[['day', 'season', 'amount']], x='day', y='amount', color='season')

In [139]:
px.histogram(df[['day', 'season', 'amount']], x='day', y=None, color='season')

In [140]:
px.histogram(df[['day', 'amount', 'hour']], x='hour', y='amount')

In [141]:
px.histogram(df[['day', 'amount', 'hour']], x='hour', y=None)

#### Columna 'title'

In [142]:
months_title_dict = dict()
for month in df['month'].unique():
    titles_for_month = set(df[df.month == month].title)
    months_title_dict[month] = titles_for_month

In [143]:
common_titles = set.intersection(*months_title_dict.values())

print("Titles common across all months:")
print(common_titles)

Titles common across all months:
set()


In [144]:
for title in df['title'].iloc[:20]:
    print(title)

COMPRA TARJ. 5402XXXXXXXX1026 562-GADIS XINZO DE LIMIA-XINZO DE LIMI
COMPRA APPLE PAY EN CONDIS BUSTAMAN. MADRID. TARJ. :*401403
767001220289 SUPER.FROIZ URUGUAY \VIGO\ES1710242023
TRANSF. - RODRIGUEZ MONTALVO MARIA NIEVES
S/ORD.TRANSF. - EOI
Mandarina ho
TRANSF. - Begona Vicente
REINTEGRO. ATM:00492211. NARON. TARJ. :*320546
Traspaso recibido Cuenta Nómina
ALQUILER MES DE DICIEMBRE
TELEFONOS TELEFONICA DE ESPANA. S.A.U. FIJO965451558.ENE
COMPRA EN RESTAURANTE BOC. A CORU¥A. TARJ. :*446134
COMPRA
COMPRAS CON TARJETA IBERCAJA-M.CHOCO ZIELO MADR
Pago en FARMACIA LDA.SUSANA RABELVERA ES
RECIBO STEP UP SCHOOL S.L. Nº RECIBO 0049 4370 755 BBBYMWC REF. MANDATO 010000000610 490
Mytaxi.com*1a6ezd
IBERIA EXPRESS
Transferencia emitida a natalia muñoz olza Cata Vinos Andres Zancada
767001220289 ESTANCO CANIDO RU.CANIDO. 227


## Apéndice

- Python 3.10.11

- Requirements:
    ```r
    asttokens==2.4.1
    attrs==23.1.0
    colorama==0.4.6
    comm==0.2.0
    debugpy==1.8.0
    decorator==5.1.1
    exceptiongroup==1.1.3
    executing==2.0.1
    fastjsonschema==2.19.0
    ipykernel==6.26.0
    ipython==8.17.2
    jedi==0.19.1
    jsonschema==4.20.0
    jsonschema-specifications==2023.11.1
    jupyter_client==8.6.0
    jupyter_core==5.5.0
    matplotlib-inline==0.1.6
    nbformat==5.9.2
    nest-asyncio==1.5.8
    numpy==1.26.2
    packaging==23.2
    pandas==2.1.3
    parso==0.8.3
    platformdirs==4.0.0
    plotly==5.18.0
    prompt-toolkit==3.0.41
    psutil==5.9.6
    pure-eval==0.2.2
    Pygments==2.16.1
    python-dateutil==2.8.2
    pytz==2023.3.post1
    pywin32==306
    pyzmq==25.1.1
    referencing==0.31.0
    rpds-py==0.13.0
    six==1.16.0
    stack-data==0.6.3
    tenacity==8.2.3
    tornado==6.3.3
    traitlets==5.13.0
    tzdata==2023.3
    wcwidth==0.2.10
    ```
